In [1]:
import json
import numpy as np
from tqdm import tqdm
from collections import Counter

In [2]:
from gensim.models import word2vec
from gensim import models
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from lexrank import LexRank

In [3]:
pre_data_PATH = './data/'

In [4]:
with open(pre_data_PATH+'com_product_intro_word_articles_test.json') as jsonfile:
    com_product_intro_word = json.load(jsonfile)
com_product_intro_word

  'KKday 事先 購買 首爾塔 門票 享受 實惠 價格 現場 櫃台 迅速 換 票 服務 有效 節省 現場 排隊 人 擠 人 時間 '],
 ['立即 預約 日本 WiFi機 Softbank 4G 吃 飽 直接 成田 羽田 機場 領取 ',
  'NRT/HND ',
  '日本 WiFi機 推薦 適合 家族 好友 畢業 旅行 多 人 連線 ',
  '參與 日本 九州 日 遊 從 福岡 搭乘 專車 走訪 山口縣 超人氣 景點    123  座 綿延 百 米 以上 赤紅 鳥居 組合 成 元 隅稻成 神社 也 美國  CNN  評選為 日本 最 美 場所 一 ',
  '有 日本 國內 第二 長 跨海 大橋 — 角島 大橋 在 廣闊 湛藍 海面 綿延 不斷 延伸 橋樑 清澈 透明 海洋 交織 壯觀 美景 不容 錯過 ',
  '預先 購買 福岡塔 觀景 門票 享有 超值 優惠 使用 電子 票券 輕鬆 登上 日本 最 高 海濱塔 你 在  116  公尺  123 公尺 高 瞭望台 環視 福岡 市景 博多灣 享受 浪漫 海濱 風景 迷人 街景 ',
  '搭乘 電車 前往 九州 福岡 最 具 代表性 旅遊 景點 太宰府 柳川 暢遊 套票 能 搭乘 柳川 觀光 列車 「 水都號 」 「 旅人號 」 還 能 在 柳川 悠閒 乘船 享受 美景 ',
  '九州 一日 遊 搭乘 福岡 出發 巴士 前往 日本 西南 地區 最 經典 景點 溫泉區 ',
  '遊逛 太宰府 天滿宮 周 遭表 參道 品嚐 當地 特色 美食 漫步 樸實 湯布院 溫泉街 遠眺 壯麗 自然 美景 別府 擁有 豐富 溫泉 所 之 處 皆 泉源 你 親眼 感受 灶 地獄 特殊 奇景 享受 舒適 放鬆 葫蘆 溫泉 '],
 ['東京 迪士尼 門票 一日券 於 舞濱站 領取 實體票 前往 東京 迪士尼 樂園 東京 迪士尼 海洋 不需 人 擠 人 排隊 購 票 KKday 販售 東京 迪士尼 門票 不 受 入園 管制 限制 實體 票券 直接 入 園 讓 比 更 輕鬆 進入 東京 迪士尼 夢幻 世界 ',
  '連接 成田 機場 上野 最 的 交通 方式 ',
  '成田 機場 上車 後 中間 停車 直達 日暮里 上野 機場 交通 靠 張 票 ',
  '立即 預約 日本 WiFi機 Softbank 4G 吃

## LexRank

### Build Graph

In [5]:
similarity_matrix_list = []
N = 20       #set up the length limit
for i in tqdm(range(len(com_product_intro_word))):
    sent_num  = len(com_product_intro_word[i])
    if sent_num >= N:
        new_sent_list = com_product_intro_word[i][0:N]
    else:
        new_sent_list = com_product_intro_word[i] + [''] * (N-sent_num)
    sent_num = N
    lxr = LexRank(new_sent_list)
    tf_scores = [Counter(sent.split()) for sent in new_sent_list]
    similarity_matrix = lxr._calculate_similarity_matrix(tf_scores)
    for j in range(len(similarity_matrix)):
        similarity_matrix[j][j] = 0
    similarity_matrix_list.append(similarity_matrix)

100%|██████████| 174/174 [00:00<00:00, 242.09it/s]


In [6]:
ranked_sentence_list = []
for i in tqdm(range(len(com_product_intro_word))):
    lex_scores = lxr.rank_sentences(com_product_intro_word[i], threshold=None)
    zipped = zip(lex_scores, com_product_intro_word[i])
    ranked_sentence = sorted(zipped, key = lambda x: x[0], reverse = True)
    ranked_sentence_list.append(ranked_sentence)
ranked_sentence_list

100%|██████████| 174/174 [00:02<00:00, 71.87it/s]


 '),
  (1.1941685035379896,
   '位於 高雄 十鼓橋 糖 文創 園區 有 百 年 文化 遺產 秉持 「 守舊 創新 」 理念 將 停產 閒置 糖倉 空間 再 造 進而 保留 台灣 糖廠 歷史 原貌 兼具 藝術 人文 內涵 自然 環境 擊樂 表演 融合 場域 為 處 充滿 文化 知性 觀光 新 景點 在  KKday  預訂 不須 現場 排隊 線上 購買 便宜 '),
  (1.0595006563380056, '現在 訂購 高雄 愛河 愛 船 船票 直接 國賓站 兌換 ， 輕鬆 搭船 遊 愛河 '),
  (1.0453993688251229,
   '位於 高雄 蓮潭 滑水 主題 樂園 於 2019年 外 開放 採用 原裝 進口 德國 Rixen  System 2 '),
  (0.9230785734432567,
   '跟著 「 小人物 玩 食堂 」 專業 老師 腳步 從 原料 您 成為 台灣 麵包 師傅 ， 親身 體驗 台式 麵包 動手 做 樂趣 '),
  (0.8694775490988286, '台灣 麵包 造型 可愛 種類 多元 現在 應用 當季 在地 好 食材 口味 獨特 '),
  (0.8380392411606731,
   '日本 鈴鹿 唯一 海外 授權 全 台 第一 座 駕駛 主題 樂園 透過 券 能 享有 現場 購 票 低價 並 體驗 各式各樣 新奇 刺激 駕駛 設施 園區 包括 「 迷你 鈴鹿 賽道 」 「 賽道 樂園 」 兩 主題 是 親子 朋友 齊聚一堂 遊玩 新 選擇 '),
  (0.8258033459340541, '南部 網美 景點 '),
  (0.797920740284104,
   '來自 澳洲  Goride 滑水 雙 系統 皆 倍受 業界 肯定 國際級 專業 設備 兼具 環保 安全 在  KKday  事先 購買 ， 訂 即 出 使用 最 方便 '),
  (0.7790091702736563,
   'i-Ride  擁有 全 世界 最 先進 體感 模擬 系統 宇劇場級 包覆性  360 全 景 視聽 饗宴 將 乘客 推進 巨大 球幕 視界 裡 雙腳 懸空 六 軸 平台 擺盪 搭配風 聲音 光影 水霧 氣味 多 種 特效 展演 打造 視覺 聽覺 嗅覺 與 移動 擬真 

### Sentence Selection

In [7]:
from functools import reduce
num = 6 # choose top n sentences
sentence_select_list = []
for s in ranked_sentence_list:
    l = min(len(s),num)
    sentence_select = [y for (x,y) in s[0:l]]
    
    sentence_select = reduce(lambda x, y:str(x)+str(y),sentence_select)
    sentence_select_list.append(sentence_select)

In [8]:
result_PATH = './data/result/'
with open(result_PATH +'lexrank.json', 'w') as outfile:
    json.dump(sentence_select_list, outfile)

In [9]:
sentence_select_list

 好 食材 回到 烹飪 教室 後 教 如何 做出 道地 泰式 料理 並 最後 細細 品嚐 完成 美食 Clay Pot  Thai  Cooking Mate  料理 教室 位在 臥 佛寺 約  50 公尺 位置 會 前往 泰國 當地 市場 認識 地 食材 回到 料理 教室 後 經由 廚師 解說 示範 帶領 你 完成 一道道 美味 泰式 料理 跟著 大廚 一起 搭乘 嘟嘟車 前往 曼谷 當地 市場 採買 食材 在 有趣 生動 教學 下 一步步 學會 烹煮出 道地 美味 泰式 料理 老師 帶領到 當地 市場 採買 食材 三 小時 完成 道 特色 料理 帶 步 一 步 完成 美味 經典 泰式 菜餚 課程 結束 後 能 來自 世界 國 學員 一起 享用 美食 廚師 您 當地 市場 認識 地 食材 烹飪 道 泰式 料理 與 來自 國 學員 饗 佳餚 最多 人 參與 小班 教學 將 老師 帶領到 當地 市場 採買 道地 香料 食材 三 小時 完成 道 特色 料理 結束 後 能 來自 世界 國 學員 一起 享用 美食 ',
 '大都會 博物館 美國 最 藝術 博物館 是 英國 倫敦 大英 博物館 法國 巴黎 盧浮宮 齊名 的 世界 大 博物館 New York  City PASS 你 最好 紐約 旅遊 夥伴 用 張 紐約 必備 通行證 暢行 大都會 博物館 古根漢 博物館 帝國 大廈 觀景臺 六 紐約 去 景點 省去 排隊 購 票 比價 時間 享受 自由自在 紐約 旅 透過  KKday  訂購 紐約 城市 景點 旅遊卡 任選  1 - 10 日 票券 自由 安排 行程 觀光 於 購買 天數 一 次 暢遊 超過  100 個 景點 您 一睹 自由 女神像 參觀 大都會 藝術 博物館 杜莎 夫人 蠟像館 911 悼念 中心 搭乘 隨上 隨下 觀光 巴士 ； 你 可 參與 在地 特色 導覽 進入 帝國 大廈 洛克斐勒 觀景台 欣賞 紐約 景色 以 最 經濟 實惠 票價 輕鬆 玩透 紐約 感受 蘋果 獨有 城市 魅力 吃到 不 降速 加拿大 墨西哥 共用  5 GB 可 選擇  7 / 8 / 12 / 15 / 30  天 方案 皆 加贈 美國 加拿大 墨西哥 夏威夷 境 無限 市話 手機 通話 另 提供 台灣 郵寄 府 服務 讓 下 飛機 後 插卡 使用 輕鬆 暢遊 美國 大 